In [211]:
import pandas as pd
import numpy as np
import pandas_ta as ta

In [212]:
data =  pd.read_csv('nifty_200/5M/ABCAPITAL.csv')

In [213]:
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime',inplace=True)

In [214]:

def calculate_macd(data, fast, slow, signal):
    data['MACD_F'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACD_{fast}_{slow}_{signal}']
    data['MACD_S'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDs_{fast}_{slow}_{signal}']
    data['MACD_H'] = data.ta.macd(fast=fast, slow=slow, signal=signal)[f'MACDh_{fast}_{slow}_{signal}']
    return data


In [215]:
def custom_resample(df, start='09:15', end='15:30', freq='1f'):
    df = df.between_time(start, end)
    return df.resample(freq, offset='0h15min').agg(
        {
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum',
        }
    )


In [216]:
df_1Hr = custom_resample(data, start='09:15', end='15:30', freq='1h')

In [217]:
df_1Hr.dropna(inplace=True)

In [218]:
df_1Hr.tail(18)

,Open,High,Low,Close,Volume
Datetime,,,,,
2024-08-07 12:15:00,205.36,206.50,205.16,206.35,498668
2024-08-07 13:15:00,206.40,209.50,205.64,209.47,760892
2024-08-07 14:15:00,209.47,212.69,209.00,211.60,2051325
2024-08-07 15:15:00,211.79,211.90,211.00,211.60,290297
2024-08-08 09:15:00,209.75,212.49,208.60,211.55,716389
2024-08-08 10:15:00,211.53,212.36,211.01,212.09,426030
2024-08-08 11:15:00,212.14,213.00,211.50,212.50,435593
2024-08-08 12:15:00,212.58,213.09,211.42,212.77,369959
2024-08-08 13:15:00,212.77,214.15,212.21,213.19,792274


In [219]:
df_15M = custom_resample(data,freq='15min')

In [220]:
df_15M.dropna(inplace=True)

In [221]:
df_15M.tail(20)

,Open,High,Low,Close,Volume
Datetime,,,,,
2024-08-09 10:30:00,210.30,210.75,210.19,210.40,20260
2024-08-09 10:45:00,210.55,211.11,210.13,210.74,24434
2024-08-09 11:00:00,210.74,211.79,210.31,211.01,81384
2024-08-09 11:15:00,211.01,211.20,210.70,210.76,17929
2024-08-09 11:30:00,210.75,210.97,210.61,210.61,16089
2024-08-09 11:45:00,210.61,212.54,210.59,212.50,150396
2024-08-09 12:00:00,212.57,212.96,211.77,212.94,96151
2024-08-09 12:15:00,212.94,212.95,211.90,212.25,163998
2024-08-09 12:30:00,212.25,212.59,211.95,212.50,39923


In [222]:
def calculate_pivot_levels(high, low, close):
    pp = (high + low + close) / 3
    tc = (pp + high) / 2
    bc = (pp + low) / 2
    r1 = 2 * pp - low
    r2 = pp + (high - low)
    r3 = high + 2 * (pp - low)
    s1 = 2 * pp - high
    s2 = pp - (high - low)
    s3 = low - 2 * (high - pp)
    
    return pp, tc, bc, r1, r2, r3, s1, s2, s3

In [223]:
def apply_pivots(df):
    # Shift OHLC columns to get yesterday's data
    df['Prev_High'] = df['High'].shift(1)
    df['Prev_Low'] = df['Low'].shift(1)
    df['Prev_Close'] = df['Close'].shift(1)

    # Apply the function to calculate pivot levels using yesterday's data
    df[['PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3']] = df.apply(
        lambda row: calculate_pivot_levels(row['Prev_High'], row['Prev_Low'], row['Prev_Close']), 
        axis=1, 
        result_type='expand'
    )
    return df


Help on method resample in module pandas.core.generic:

resample(rule, axis: 'Axis | lib.NoDefault' = <no_default>, closed: "Literal['right', 'left'] | None" = None, label: "Literal['right', 'left'] | None" = None, convention: "Literal['start', 'end', 's', 'e'] | lib.NoDefault" = <no_default>, kind: "Literal['timestamp', 'period'] | None | lib.NoDefault" = <no_default>, on: 'Level | None' = None, level: 'Level | None' = None, origin: 'str | TimestampConvertibleTypes' = 'start_day', offset: 'TimedeltaConvertibleTypes | None' = None, group_keys: 'bool_t' = False) -> 'Resampler' method of pandas.core.frame.DataFrame instance
    Resample time-series data.

    Convenience method for frequency conversion and resampling of time series.
    The object must have a datetime-like index (`DatetimeIndex`, `PeriodIndex`,
    or `TimedeltaIndex`), or the caller must pass the label of a datetime-like
    series/index to the ``on``/``level`` keyword parameter.

    Parameters
    ----------
    rule 

In [225]:
df_daily = custom_resample(data,freq='D').dropna()

In [226]:
df_daily = apply_pivots(df_daily)

In [227]:
df_daily

,Open,High,Low,Close,Volume,Prev_High,Prev_Low,Prev_Close,PP,TC,BC,R1,R2,R3,S1,S2,S3
Datetime,,,,,,,,,,,,,,,,,
2020-07-03 00:15:00,67.85,68.00,65.10,65.60,3021321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-06 00:15:00,66.00,66.70,65.15,65.65,1937998,68.00,65.10,65.60,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667
2020-07-07 00:15:00,65.80,67.10,64.50,66.35,3933601,66.70,65.15,65.65,65.833333,66.266667,65.491667,66.516667,67.383333,68.066667,64.966667,64.283333,63.416667
2020-07-08 00:15:00,66.55,66.95,64.15,64.45,2792572,67.10,64.50,66.35,65.983333,66.541667,65.241667,67.466667,68.583333,70.066667,64.866667,63.383333,62.266667
2020-07-09 00:15:00,64.60,65.25,63.75,64.15,1433452,66.95,64.15,64.45,65.183333,66.066667,64.666667,66.216667,67.983333,69.016667,63.416667,62.383333,60.616667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05 00:15:00,204.50,209.39,200.05,205.00,11553471,217.15,208.00,211.30,212.150000,214.650000,210.075000,216.300000,221.300000,225.450000,207.150000,203.000000,198.000000
2024-08-06 00:15:00,207.75,210.90,202.64,203.60,5492316,209.39,200.05,205.00,204.813333,207.101667,202.431667,209.576667,214.153333,218.916667,200.236667,195.473333,190.896667
2024-08-07 00:15:00,206.21,212.69,203.00,211.60,5824918,210.90,202.64,203.60,205.713333,208.306667,204.176667,208.786667,213.973333,217.046667,200.526667,197.453333,192.266667


In [228]:
data['Date'] = pd.to_datetime(data.index).date

In [229]:
data

,Open,High,Low,Close,Volume,Date
Datetime,,,,,,
2020-07-03 09:15:00,67.85,68.00,67.45,67.45,223536,2020-07-03
2020-07-03 09:20:00,67.40,68.00,67.15,67.90,153437,2020-07-03
2020-07-03 09:25:00,67.95,68.00,67.40,67.60,135082,2020-07-03
2020-07-03 09:30:00,67.50,67.70,67.15,67.20,89835,2020-07-03
2020-07-03 09:35:00,67.30,67.40,66.50,66.65,166271,2020-07-03
...,...,...,...,...,...,...
2024-08-09 15:05:00,212.53,212.55,212.37,212.55,14511,2024-08-09
2024-08-09 15:10:00,212.64,212.75,211.80,211.92,82129,2024-08-09
2024-08-09 15:15:00,211.83,211.89,211.00,211.53,61012,2024-08-09


In [230]:
df_15M['VWMA'] = df_15M.ta.vwma(length=10)

In [231]:
df_15M.dropna(inplace=True)

In [232]:
df_15M

,Open,High,Low,Close,Volume,VWMA
Datetime,,,,,,
2020-07-03 11:30:00,66.30,66.60,66.30,66.45,48703,66.810771
2020-07-03 11:45:00,66.55,66.65,66.40,66.40,69892,66.471869
2020-07-03 12:00:00,66.35,66.60,66.30,66.50,66568,66.526839
2020-07-03 12:15:00,66.50,66.75,66.40,66.55,52905,66.451999
2020-07-03 12:30:00,66.55,66.70,66.25,66.40,60401,66.406065
...,...,...,...,...,...,...
2024-08-09 14:15:00,213.15,213.36,212.21,212.61,50647,212.703230
2024-08-09 14:30:00,212.61,212.66,212.08,212.36,47337,212.658435
2024-08-09 14:45:00,212.36,212.90,212.23,212.84,67107,212.760714


In [233]:
data['VWMA_15M'] = df_15M['VWMA'].reindex(data.index,method ='ffill')

In [234]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 76121 entries, 2020-07-03 09:15:00 to 2024-08-09 15:25:00
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Open      76121 non-null  float64
 1   High      76121 non-null  float64
 2   Low       76121 non-null  float64
 3   Close     76121 non-null  float64
 4   Volume    76121 non-null  int64  
 5   Date      76121 non-null  object 
 6   VWMA_15M  76094 non-null  float64
dtypes: float64(5), int64(1), object(1)
memory usage: 4.6+ MB


In [235]:
df_15M['RSI'] = ta.rsi(close =df_15M['Close'], length=21)

In [236]:
df_15M

,Open,High,Low,Close,Volume,VWMA,RSI
Datetime,,,,,,,
2020-07-03 11:30:00,66.30,66.60,66.30,66.45,48703,66.810771,NaN
2020-07-03 11:45:00,66.55,66.65,66.40,66.40,69892,66.471869,NaN
2020-07-03 12:00:00,66.35,66.60,66.30,66.50,66568,66.526839,NaN
2020-07-03 12:15:00,66.50,66.75,66.40,66.55,52905,66.451999,NaN
2020-07-03 12:30:00,66.55,66.70,66.25,66.40,60401,66.406065,NaN
...,...,...,...,...,...,...,...
2024-08-09 14:15:00,213.15,213.36,212.21,212.61,50647,212.703230,55.268355
2024-08-09 14:30:00,212.61,212.66,212.08,212.36,47337,212.658435,53.863330
2024-08-09 14:45:00,212.36,212.90,212.23,212.84,67107,212.760714,56.112582


In [237]:
df_15M['RSI_Signal'] = np.where((df_15M['RSI'] > 60) & (df_15M['RSI'].shift(1) < 60), -1,
                        np.where((df_15M['RSI'] < 40) & (df_15M['RSI'].shift(1) > 40), 1, 0))

In [238]:
df_15M[df_15M['RSI_Signal'] == 1]

,Open,High,Low,Close,Volume,VWMA,RSI,RSI_Signal
Datetime,,,,,,,,
2020-07-06 14:30:00,65.65,65.80,65.15,65.30,110453,65.781384,37.212613,1
2020-07-07 11:30:00,65.35,65.35,64.50,64.90,101828,65.989849,36.433564,1
2020-07-07 13:45:00,65.00,65.15,64.90,64.90,45714,64.961735,39.854147,1
2020-07-08 14:30:00,65.65,65.75,64.50,64.55,287551,65.468034,36.206341,1
2020-07-09 10:30:00,64.80,64.80,64.15,64.30,105411,64.641911,37.693988,1
...,...,...,...,...,...,...,...,...
2024-07-23 13:30:00,213.49,213.50,211.69,212.26,86234,212.460699,39.357522,1
2024-07-25 09:45:00,214.14,214.64,212.71,212.84,164966,214.508372,39.688252,1
2024-08-01 14:15:00,224.62,224.66,222.51,222.98,1094571,224.370107,36.675817,1


In [239]:
data['15M_RSI_Signal'] = df_15M['RSI_Signal'].reindex(data.index,method ='ffill').dropna()

In [240]:
data['VWMA_10'] = ta.vwma(data['Close'],volume=data['Volume'], length=10)
data['Supertrend'] = data.ta.supertrend(length=10, multiplier=4)['SUPERT_10_4.0']


In [242]:
df_daily['Date'] =  df_daily.index.date

In [243]:
df_daily

,Open,High,Low,Close,Volume,Prev_High,Prev_Low,Prev_Close,PP,TC,BC,R1,R2,R3,S1,S2,S3,Date
Datetime,,,,,,,,,,,,,,,,,,
2020-07-03 00:15:00,67.85,68.00,65.10,65.60,3021321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-03
2020-07-06 00:15:00,66.00,66.70,65.15,65.65,1937998,68.00,65.10,65.60,66.233333,67.116667,65.666667,67.366667,69.133333,70.266667,64.466667,63.333333,61.566667,2020-07-06
2020-07-07 00:15:00,65.80,67.10,64.50,66.35,3933601,66.70,65.15,65.65,65.833333,66.266667,65.491667,66.516667,67.383333,68.066667,64.966667,64.283333,63.416667,2020-07-07
2020-07-08 00:15:00,66.55,66.95,64.15,64.45,2792572,67.10,64.50,66.35,65.983333,66.541667,65.241667,67.466667,68.583333,70.066667,64.866667,63.383333,62.266667,2020-07-08
2020-07-09 00:15:00,64.60,65.25,63.75,64.15,1433452,66.95,64.15,64.45,65.183333,66.066667,64.666667,66.216667,67.983333,69.016667,63.416667,62.383333,60.616667,2020-07-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-05 00:15:00,204.50,209.39,200.05,205.00,11553471,217.15,208.00,211.30,212.150000,214.650000,210.075000,216.300000,221.300000,225.450000,207.150000,203.000000,198.000000,2024-08-05
2024-08-06 00:15:00,207.75,210.90,202.64,203.60,5492316,209.39,200.05,205.00,204.813333,207.101667,202.431667,209.576667,214.153333,218.916667,200.236667,195.473333,190.896667,2024-08-06
2024-08-07 00:15:00,206.21,212.69,203.00,211.60,5824918,210.90,202.64,203.60,205.713333,208.306667,204.176667,208.786667,213.973333,217.046667,200.526667,197.453333,192.266667,2024-08-07


In [244]:
data['Date']

Datetime
2020-07-03 09:15:00    2020-07-03
2020-07-03 09:20:00    2020-07-03
2020-07-03 09:25:00    2020-07-03
2020-07-03 09:30:00    2020-07-03
2020-07-03 09:35:00    2020-07-03
                          ...    
2024-08-09 15:05:00    2024-08-09
2024-08-09 15:10:00    2024-08-09
2024-08-09 15:15:00    2024-08-09
2024-08-09 15:20:00    2024-08-09
2024-08-09 15:25:00    2024-08-09
Name: Date, Length: 76121, dtype: object

In [246]:
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | AnyArrayLike | None' = None, left_on: 'IndexLabel | AnyArrayLike | None' = None, right_on: 'IndexLabel | AnyArrayLike | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.

    A named Series object is treated as a DataFrame with a single named column.

    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.

  

In [247]:
data_with_pivot = pd.merge(data, df_daily[['Date', 'PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3']], on='Date', how='left')

In [256]:
data_with_pivot[pd.isna(data_with_pivot['PP'])]['Date'].unique()

array([datetime.date(2020, 7, 3), datetime.date(2020, 11, 14),
       datetime.date(2021, 11, 4), datetime.date(2022, 10, 24),
       datetime.date(2023, 11, 12)], dtype=object)

In [257]:
df_daily[pd.isna(df_daily['PP'])]

,Open,High,Low,Close,Volume,Prev_High,Prev_Low,Prev_Close,PP,TC,BC,R1,R2,R3,S1,S2,S3,Date
Datetime,,,,,,,,,,,,,,,,,,
2020-07-03 00:15:00,67.85,68.0,65.1,65.6,3021321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-03
